# Data Processing

## Importing the libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Importing the dataset

In [2]:
dataset = pd.read_table("data/airfoil_self_noise.dat", 
                        header=None, 
                        names=['frequency', 'angle_of_attack', 'chord_length', 'free_stream_velocity', 
                               'suction_side_displacement_thickness', 'scaled_sound_pressure_level'])
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1].values

In [3]:
X

array([[8.00000e+02, 0.00000e+00, 3.04800e-01, 7.13000e+01, 2.66337e-03],
       [1.00000e+03, 0.00000e+00, 3.04800e-01, 7.13000e+01, 2.66337e-03],
       [1.25000e+03, 0.00000e+00, 3.04800e-01, 7.13000e+01, 2.66337e-03],
       ...,
       [4.00000e+03, 1.56000e+01, 1.01600e-01, 3.96000e+01, 5.28487e-02],
       [5.00000e+03, 1.56000e+01, 1.01600e-01, 3.96000e+01, 5.28487e-02],
       [6.30000e+03, 1.56000e+01, 1.01600e-01, 3.96000e+01, 5.28487e-02]])

In [4]:
y

array([126.201, 125.201, 125.951, ..., 106.604, 106.224, 104.204])

## Feature Scaling

In [5]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X = sc.fit_transform(X)

## Splitting the dataset into the Training set and Test set

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

## Training 

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators': [10, 100, 1000],
    'criterion': ['mse', 'mae'],
    'max_features': [None, 'auto', 'sqrt', 'log2'],
    'bootstrap': [True, False],
    'random_state': [0]
}

regressors = GridSearchCV(estimator=RandomForestRegressor(), param_grid=params, verbose=10, n_jobs=-1)
regressors.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   17.0s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:   21.8s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:   55.1s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:  1

GridSearchCV(estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'criterion': ['mse', 'mae'],
                         'max_features': [None, 'auto', 'sqrt', 'log2'],
                         'n_estimators': [10, 100, 1000], 'random_state': [0]},
             verbose=10)

## Predicting the Test set results

In [8]:
regressor = regressors.best_estimator_
y_pred = regressor.predict(X_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[131.95 130.79]
 [118.82 119.54]
 [111.08 110.32]
 [116.69 117.4 ]
 [126.8  127.62]
 [122.03 121.66]
 [118.91 118.69]
 [127.55 130.03]
 [115.29 114.73]
 [125.12 124.21]
 [125.85 125.84]
 [132.61 131.52]
 [130.19 130.7 ]
 [118.91 117.81]
 [126.08 126.66]
 [115.71 113.14]
 [127.93 126.76]
 [133.9  133.38]
 [115.72 118.08]
 [116.68 116.15]
 [123.35 123.46]
 [130.35 130.09]
 [129.06 129.93]
 [125.53 120.66]
 [131.97 134.06]
 [121.82 122.09]
 [114.02 117.09]
 [138.83 138.76]
 [129.51 130.83]
 [130.2  128.24]
 [129.46 128.95]
 [123.63 122.53]
 [105.7  103.38]
 [126.77 126.41]
 [133.18 131.8 ]
 [130.63 130.96]
 [116.77 119.25]
 [112.84 109.64]
 [126.73 127.78]
 [127.16 129.01]
 [132.24 132.54]
 [124.32 125.48]
 [134.08 138.27]
 [122.57 123.13]
 [112.42 111.03]
 [134.09 135.96]
 [126.45 127.12]
 [128.9  129.67]
 [128.17 125.65]
 [127.29 129.24]
 [124.23 123.21]
 [124.69 127.  ]
 [122.2  119.91]
 [127.06 126.56]
 [121.25 121.53]
 [121.86 122.23]
 [116.45 113.3 ]
 [130.85 128.34]
 [121.56 121.7

## Measuring performance

In [9]:
from sklearn.metrics import r2_score 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error

In [10]:
r2_score(y_test, y_pred)

0.9405274454363514

In [11]:
mean_squared_error(y_test, y_pred)

2.821030156108962

In [12]:
median_absolute_error(y_test, y_pred)

0.9402520000007399